In [ ]:
import tensorflow as tf
import tensorflow_probability as tfp

Markov models are predictive chain models that allow us to predict one thing based on 1) our knowledge of the probability of that specific thing (Transition Probability) and a somewhat related phenomenon (Emission probability). The common examples to explain this usually involve predicting the weather from the current weather state and a related phenomenon (Bill mostly takes a walk on sunny days).


In [ ]:
tfd = tfp.distributions


When we think about the possible states the weather can take on, the most simple way to describe it would to be 2 separate states (Sunny and Rainy). But to describe the cycle between these states requires more. It requires a description of the emission probability of values that might be observed (observation_Pr) during each state, and a description of the probability of transitioning between both states (transition_Pr), and a description of the state that it happens to currently be in (initial_Pr). So understanding the descriptions themselves informs us of how to construct them. The initial_Pr can only have two possible values, the data is categorical, therefore we construct a Categorical distribution object. Similarly, the transition_Pr can only have 4 possible states, so the data is also categorical. The emission property (in this instance) has been deemed to be continuous and so we will create a normal distribution object for that.

In [ ]:
initial_Pr = tfd.Categorical(probs=[0.8, 0.2])
#initially we create a categorical distribution object for rainy [0], and sunny [1]
transition_Pr = tfd.Categorical(probs=[[0.7, 0.3], [0.2, 0.8]])
#because there is only 2 states, and only the possiblity of remaining the same or changing
#to the next state, we can describe the transition probability with a 2 x 2 array
# here a rainy day has a 70% chance of being followed by another rainy day
observation_Pr = tfd.Normal(loc=[0.0, 15.0], scale=[5.0, 10.0])
#here the args loc = mean value for specific state, and scale = stdev for corresponding state

Instantiate a model

In [ ]:
markov_model = tfd.HiddenMarkovModel(initial_distribution=initial_Pr, 
                                     transition_distribution=transition_Pr,
                                     observation_distribution=observation_Pr, 
                                     num_steps=7)
# num_steps = how many cycles/days we want to predict for 

So usually here, especially in sklearn is where we .fit() and .predict off the model. But tensorflow is different (in this instance). Some models are partially defined tensors (graphs). To run them requires a Session. When we are looking to use this model, the stat we are concerned with seeing is the mean probability. So we apply a .mean() function to the model, and then use a tensorflow session to evaluate the model/equation.

In [ ]:
result = markov_model.mean()



Use context management to create and run Session object.

In [ ]:
with tf.compat.v1.Session() as sess:
  print(result.numpy())
# each tensor object has built in function, .numpy() that returns the values

[2.9999998 5.9999995 7.4999995 8.25      8.625001  8.812501  8.90625  ]


In [ ]:
with tf.compat.v1.Session() as sess:
  print(result.numpy)
#printing the function itself also gives info on the shape and state of the tensor

<bound method _EagerTensorBase.numpy of <tf.Tensor: shape=(7,), dtype=float32, numpy=
array([2.9999998, 5.9999995, 7.4999995, 8.25     , 8.625001 , 8.812501 ,
       8.90625  ], dtype=float32)>>
